In [1]:
import json

In [2]:
documents = json.load(open("data/raw/chat_export.json"))

In [17]:
documents.keys()

dict_keys(['name', 'type', 'id', 'messages'])

In [22]:
documents["messages"][0], documents["messages"][2], documents["messages"][1111]

({'id': 1,
  'type': 'service',
  'date': '2016-11-04T21:06:17',
  'date_unixtime': '1478279177',
  'actor': 'Natural Language Processing',
  'actor_id': 'channel1095835958',
  'action': 'migrate_from_group',
  'title': 'Natural Language Processing',
  'text': '',
  'text_entities': []},
 {'id': 3,
  'type': 'message',
  'date': '2016-12-21T04:05:54',
  'date_unixtime': '1482278754',
  'edited': '2016-12-21T04:06:43',
  'edited_unixtime': '1482278803',
  'from': 'Dima Veselov',
  'from_id': 'user216024521',
  'text': ['Для интересующихся:  прогнал natasha через тесты factRuEval-2016 (первая дорожка, сущности типа персона, организация, гео) и получил более-менее приемлимые результаты (при loc=locorg, overall P=0.81, R=0.52, F1=0.6348)\nМожно всё это повторить, совершенно бесплатно, без смс и регистрации: ',
   {'type': 'link',
    'text': 'https://gist.github.com/dveselov/af9bc55b4be16fe1fe92d4b5347f1027'},
   ''],
  'text_entities': [{'type': 'plain',
    'text': 'Для интересующихся:  

In [16]:
relevant_keys = ["id", "date", "from", "from_id", "text"]
messages = [
    {key: message[key] for key in relevant_keys}
    for message in documents["messages"]
    if message["type"] == "message"
]
messages[35], len(messages)

({'id': 41,
  'date': '2017-01-31T16:07:54',
  'from': 'Dima Veselov',
  'from_id': 'user216024521',
  'text': [{'type': 'link',
    'text': 'https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct'}]},
 83796)

83k messages! not that much, but still significant.


#### Types of formatting applied to data

I guess we can just parse all "text" blocks into one message


In [24]:
from collections import Counter

Counter(
    part["type"] if isinstance(part, dict) else "txt"
    for message in messages
    for part in message["text"]
)

Counter({'txt': 7147761,
         'link': 8934,
         'mention_name': 1539,
         'text_link': 1441,
         'bold': 1193,
         'mention': 1072,
         'code': 989,
         'pre': 553,
         'italic': 310,
         'hashtag': 289,
         'custom_emoji': 160,
         'bot_command': 98,
         'strikethrough': 48,
         'blockquote': 46,
         'spoiler': 35,
         'underline': 21,
         'email': 16,
         'phone': 12,
         'bank_card': 9,
         'cashtag': 4})

In [41]:
texts = []

for message in messages:
    message_text = ""

    for part in message["text"]:
        if isinstance(part, dict):
            message_text += part["text"]
        else:
            message_text += part

    texts.append(message_text)

texts[0]

'Для интересующихся:  прогнал natasha через тесты factRuEval-2016 (первая дорожка, сущности типа персона, организация, гео) и получил более-менее приемлимые результаты (при loc=locorg, overall P=0.81, R=0.52, F1=0.6348)\nМожно всё это повторить, совершенно бесплатно, без смс и регистрации: https://gist.github.com/dveselov/af9bc55b4be16fe1fe92d4b5347f1027'

In [46]:
texts[35]

'https://pymorphy2.readthedocs.io/en/latest/user/guide.html#select-correct'

In [48]:
import pandas as pd

message_df = pd.DataFrame(messages)
message_df

,id,date,from,from_id,text
0,3,2016-12-21T04:05:54,Dima Veselov,user216024521,[Для интересующихся: прогнал natasha через те...
1,4,2016-12-29T13:19:06,Dima Veselov,user216024521,
2,5,2016-12-29T13:19:10,Dima Veselov,user216024521,"Чуть-чуть допилил правила, теперь результаты т..."
3,8,2017-01-29T14:37:07,Vlad Pastushenko,user133814935,"Ребят, подскажите тупню, как из текста вытащит..."
4,9,2017-01-29T17:19:12,Dima Veselov,user216024521,[Привет. Можно переиспользовать существующие г...
...,...,...,...,...,...
83791,103039,2024-06-16T00:43:10,Alexander,user264444661,"Друзья, если еще будут рекомендации - то отпиш..."
83792,103040,2024-06-16T00:43:14,Степан Долженко,user55624230,"Сори за офтоп, но интересно, есть ли библиотек..."
83793,103041,2024-06-16T00:43:21,Kristaller,user362918197,Ну англоязычные слова она точно понимает
83794,103042,2024-06-16T00:46:36,Степан Долженко,user55624230,У меня задача банально простая — книжку разбив...


In [53]:
message_df["text"] = texts
message_df["text"].iloc[0]

'Для интересующихся:  прогнал natasha через тесты factRuEval-2016 (первая дорожка, сущности типа персона, организация, гео) и получил более-менее приемлимые результаты (при loc=locorg, overall P=0.81, R=0.52, F1=0.6348)\nМожно всё это повторить, совершенно бесплатно, без смс и регистрации: https://gist.github.com/dveselov/af9bc55b4be16fe1fe92d4b5347f1027'

In [55]:
message_df.to_csv("data/prepared/messages.csv", index=False)